In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

agosto13 = pd.read_csv('1308.csv')
septiembre13 = pd.read_csv('1309.csv')
octubre13 = pd.read_csv('1310.csv')
noviembre13 = pd.read_csv('1311.csv')
diciembre13 = pd.read_csv('1312.csv')
enero14 = pd.read_csv('1401.csv')
febrero14 = pd.read_csv('1402.csv')
marzo14 = pd.read_csv('1403.csv')
abril14 = pd.read_csv('1404.csv')
mayo14 = pd.read_csv('1405.csv')
junio14 = pd.read_csv('1406.csv')
enero15 = pd.read_csv('1501.csv')
agosto15 = pd.read_csv('1508.csv')
marzo16 = pd.read_csv('1603.csv')
abril16 = pd.read_csv('1604.csv')
mayo16 = pd.read_csv('1605.csv')
junio16 = pd.read_csv('1606.csv')
enero17 = pd.read_csv('1701.csv')
agosto17 = pd.read_csv('1708.csv')

datos = pd.concat([agosto13, septiembre13, octubre13, noviembre13, diciembre13, enero14, febrero14, \
                          marzo14, abril14, mayo14, junio14, enero15, agosto15, marzo16, abril16, mayo16, \
                          junio16, enero17, agosto17])

datosForTesting = pd.read_csv('properati_dataset_testing_noprice.csv')
 

In [2]:
datosForTesting.count()

id                         14166
created_on                 14166
property_type              14166
operation                  14166
place_name                 14166
place_with_parent_names    14166
country_name               14166
state_name                 14166
lat-lon                    10487
lat                        10487
lon                        10487
surface_total_in_m2        11853
surface_covered_in_m2      13005
floor                       1368
rooms                       7500
expenses                    2543
description                14166
dtype: int64

Algunos primeros datos del set a testear. Vemos que los únicos campos con bajo porcentaje de datos son el piso y las expensas.

In [3]:
datos = datos.loc[:, ['id', 'created_on',  'property_type', 'operation', 'place_name', \
                      'place_with_parent_names', 'country_name', 'state_name', 'lat-lon', 'lat', 'lon', \
                     'surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms', 'expenses', 'description',\
                     'price_aprox_usd']]
datos.columns

Index([u'id', u'created_on', u'property_type', u'operation', u'place_name',
       u'place_with_parent_names', u'country_name', u'state_name', u'lat-lon',
       u'lat', u'lon', u'surface_total_in_m2', u'surface_covered_in_m2',
       u'floor', u'rooms', u'expenses', u'description', u'price_aprox_usd'],
      dtype='object')

In [4]:
print datosForTesting.operation.unique()

['venta' 'alquiler']


In [5]:
datosForTesting.loc[datosForTesting.operation == 'alquiler','id'].count()

1

In [6]:
print datos.operation.unique()

['sell']


In [7]:
datos['operation'] = datos['operation'].map(lambda x: 'venta' if x == 'sell' else 'alquiler')
datos.loc[datos.operation == 'venta', 'id'].count()

394859

In [8]:
print datosForTesting.property_type.unique()

['departamento' 'ph' 'casa']


In [9]:
print datos.property_type.unique()

['house' 'apartment' 'PH' 'store']


In [10]:
datos = datos.loc[datos.property_type != 'store', :]
datos.count()

id                          382226
created_on                 1101908
property_type              1101908
operation                  1101908
place_name                 1101851
place_with_parent_names    1101908
country_name                382226
state_name                  382226
lat-lon                     921414
lat                         921414
lon                         921403
surface_total_in_m2         254876
surface_covered_in_m2       307965
floor                       150100
rooms                       781325
expenses                     53797
description                 382210
price_aprox_usd             918368
dtype: int64

In [11]:
def traducirTipoPropiedad(x):
    tipo = ''
    if (x == 'house'):
        tipo = 'casa' 
    else:
        if (x == 'apartment'): 
            tipo = 'departamento'
        else: 
            if (x == 'PH'): 
                tipo = 'ph'
    return tipo        

In [12]:
datos['property_type'] = datos['property_type'].map(traducirTipoPropiedad)
datos.loc[datos.property_type == 'casa','id'].count()

144190

In [13]:
print datosForTesting.surface_total_in_m2.min()

0.0


In [14]:
print datosForTesting.surface_total_in_m2.max()

148000.0


In [15]:
print datos.loc[datos.surface_total_in_m2 > 150000, 'id'].count()

10


In [16]:
print datosForTesting.state_name.unique()

['Capital Federal' 'Buenos Aires Interior' 'Bs.As. G.B.A. Zona Sur'
 'Bs.As. G.B.A. Zona Oeste' 'Bs.As. G.B.A. Zona Norte']


In [17]:
print datosForTesting.loc[datosForTesting.state_name == 'Buenos Aires Interior', 'id'].count()

1


In [18]:
print datos.state_name.unique()

[nan 'Buenos Aires Interior' 'R\xc3\xado Negro' 'Capital Federal'
 'Buenos Aires Costa Atl\xc3\xa1ntica' 'Neuqu\xc3\xa9n' 'San Juan'
 'Bs.As. G.B.A. Zona Norte' 'Bs.As. G.B.A. Zona Sur'
 'Bs.As. G.B.A. Zona Oeste' 'Misiones' 'Santa Fe' 'San Luis'
 'C\xc3\xb3rdoba' 'Entre R\xc3\xados' 'La Pampa' 'Salta' 'Tucum\xc3\xa1n'
 'Mendoza' 'Corrientes' 'Chaco' 'Jujuy' 'Catamarca' 'La Rioja' 'Chubut'
 'Formosa' 'Tierra Del Fuego' 'Santa Cruz' 'Santiago Del Estero']


In [19]:
datos = datos.loc[(datos['state_name'].str.contains('Capital Federal')| \
                   datos['state_name'].str.contains('Bs.As. G.B.A. Zona Sur')|\
                  datos['state_name'].str.contains('Bs.As. G.B.A. Zona Oeste')|\
                  datos['state_name'].str.contains('Bs.As. G.B.A. Zona Norte')),:]
datos.count()

id                         266059
created_on                 266059
property_type              266059
operation                  266059
place_name                 266002
place_with_parent_names    266059
country_name               266059
state_name                 266059
lat-lon                    183538
lat                        183538
lon                        183538
surface_total_in_m2        189874
surface_covered_in_m2      230103
floor                       34239
rooms                      139224
expenses                    32679
description                266048
price_aprox_usd            247383
dtype: int64

In [20]:
datos = datos.loc[datos['price_aprox_usd'].notnull(),:]
datos.count()

id                         247383
created_on                 247383
property_type              247383
operation                  247383
place_name                 247326
place_with_parent_names    247383
country_name               247383
state_name                 247383
lat-lon                    170744
lat                        170744
lon                        170744
surface_total_in_m2        176942
surface_covered_in_m2      216950
floor                       33495
rooms                      129220
expenses                    31786
description                247372
price_aprox_usd            247383
dtype: int64

In [21]:
#Set de entrenamiento
datosFiltrado = datos.loc[:, ['lat', 'lon', 'surface_total_in_m2', 'surface_covered_in_m2', 'price_aprox_usd']]
datosFiltrado = datosFiltrado.loc[datosFiltrado['lat'].notnull(),:]
datosFiltrado = datosFiltrado.loc[datosFiltrado['lon'].notnull(),:]
datosFiltrado = datosFiltrado.loc[datosFiltrado['surface_total_in_m2'].notnull(),:]
datosFiltrado = datosFiltrado.loc[datosFiltrado['surface_covered_in_m2'].notnull(),:]
datosFiltrado = datosFiltrado.loc[datosFiltrado['price_aprox_usd'].notnull(),:]

In [22]:
#Set de validacion
prueba = datos.loc[datos['place_name'] == 'Palermo',['lat', 'lon', 'surface_total_in_m2', 'surface_covered_in_m2', 'price_aprox_usd']].head(15)
prueba = prueba.loc[prueba['lat'].notnull(),:]
prueba = prueba.loc[prueba['lon'].notnull(),:]
prueba = prueba.loc[prueba['surface_total_in_m2'].notnull(),:]
prueba = prueba.loc[prueba['surface_covered_in_m2'].notnull(),:]
prueba = prueba.loc[prueba['price_aprox_usd'].notnull(),:]
prueba['price_aprox_usd']

293      88000.0
314     650000.0
1414    165000.0
1959    112000.0
1960    114000.0
1962    116000.0
1963    118000.0
1964    120000.0
2148     87550.0
2149     82400.0
2150     82400.0
2151     87550.0
2205    103000.0
Name: price_aprox_usd, dtype: float64

In [23]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(datosFiltrado, datosFiltrado['price_aprox_usd'], test_size=0.2)

/home/leandro/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [25]:
#Prueba Random forest Regressor
#importacion librerias ML
from sklearn.ensemble import RandomForestRegressor

modelRF = RandomForestRegressor(n_estimators=100, max_features='sqrt')
modelRF.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [26]:
print(modelRF.predict(prueba))

[  88372.      649539.99    165007.68    112273.3944  114001.77
  115411.1231  118130.      119771.3299   87115.22     82357.39     82357.39
   87425.39    103043.1033]


In [27]:
#Prueba KNeioghborth Regressor
from sklearn.neighbors import KNeighborsRegressor

modelKNR = KNeighborsRegressor()
modelKNR.fit(x_train,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [28]:
print(modelKNR.predict(prueba))

[  88000.   650000.   165000.   112000.   114000.   116000.   118000.
  120000.    87513.2   82400.    82400.    87513.2  103000. ]


In [31]:
from sklearn.neighbors import RadiusNeighborsRegressor
modelRNR = RadiusNeighborsRegressor()
modelRNR.fit(x_train,y_train)

RadiusNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
             metric_params=None, p=2, radius=1.0, weights='uniform')

In [32]:
print(modelRNR.predict(prueba))

[  88000.  650000.  165000.  112000.  114000.  116000.  118000.  120000.
   87550.   82400.   82400.   87550.  103000.]


In [29]:
#Prueba LinearRegression
#http://scikit-learn.org/stable/modules/linear_model.html#ridge-regression
    
from sklearn.linear_model import LinearRegression, LogisticRegression

modelLinR = LinearRegression()
modelLinR.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [30]:
print(modelLinR.predict(prueba))

[  88000.  650000.  165000.  112000.  114000.  116000.  118000.  120000.
   87550.   82400.   82400.   87550.  103000.]


In [35]:
#Prueba RidgeRegression
from sklearn import linear_model
modelRid = linear_model.Ridge (alpha = .5)
modelRid.fit (x_train,y_train) 

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [36]:
print(modelRid.predict(prueba))

[  88000.  650000.  165000.  112000.  114000.  116000.  118000.  120000.
   87550.   82400.   82400.   87550.  103000.]


In [37]:
#Prueba Lasso
modelLas = linear_model.Lasso (alpha = 0.1)
modelLas.fit (x_train,y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [38]:
print(modelLas.predict(prueba))

[  88000.00004462  650000.00007356  165000.00005442  112000.00005036
  114000.0000508   116000.00005123  118000.00005166  120000.0000521
   87550.00004878   82400.00004872   82400.00004872   87550.00004878
  103000.0000516 ]


In [39]:
#Prueba LassoLars
modelLar = linear_model.LassoLars (alpha = 0.1)
modelLar.fit (x_train,y_train)

LassoLars(alpha=0.1, copy_X=True, eps=2.2204460492503131e-16,
     fit_intercept=True, fit_path=True, max_iter=500, normalize=True,
     positive=False, precompute='auto', verbose=False)

In [40]:
print(modelLar.predict(prueba))

[  88013.547415    649959.78539103  165006.18144375  112011.25152786
  114011.06020393  116010.86888     118010.67755607  120010.48623214
   87563.59046289   82414.083122     82414.083122     87563.59046289
  103012.11248554]


In [41]:
#Prueba Bayesian
#http://scikit-learn.org/stable/modules/linear_model.html#bayesian-regression
modelBay = linear_model.BayesianRidge()
modelBay.fit(x_train,y_train)

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)

In [42]:
print(modelBay.predict(prueba))

[  87999.99999963  649999.99999992  164999.99999971  111999.99999972
  113999.99999972  115999.99999972  117999.99999972  119999.99999972
   87549.99999977   82399.99999976   82399.99999976   87549.99999977
  102999.99999967]


In [31]:
#Prueba LogisticRegression
#http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
modelLogR = LogisticRegression()
modelLogR.fit(x_train,y_train)

ValueError: Unknown label type: 'continuous'

In [24]:
#Prueba LinearSVR
#http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVR.html#sklearn.svm.LinearSVR
from sklearn import svm
modelSVR = svm.LinearSVR()
modelSVR.fit(x_train,y_train) 

LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0)

In [25]:
print(modelSVR.predict(prueba))

[  87999.99999543  649999.99996641  164999.99999145  111999.99999419
  113999.99999408  115999.99999398  117999.99999388  119999.99999377
   87549.99999545   82399.99999571   82399.99999571   87549.99999545
  102999.99999465]


In [35]:
#Prueba DecisionTree
#http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor

from sklearn import tree
modelDTR = tree.DecisionTreeRegressor()
modelDTR = modelDTR.fit(x_train, y_train)

In [36]:
print(modelDTR.predict(prueba))

[  88000.  650000.  165000.  112000.  114000.  116000.  118000.  120000.
   87550.   82400.   82400.   87550.  103000.]


In [39]:
#Prueba DecisionTree
#http://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html#sklearn.gaussian_process.GaussianProcessRegressor

from sklearn import gaussian_process
modelGP = gaussian_process.GaussianProcessRegressor()
modelGP = modelGP.fit(x_train, y_train)

MemoryError: 

In [40]:
#http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor
from sklearn import neural_network
modelNN = neural_network.MLPRegressor()
modelNN = modelNN.fit(x_train, y_train)

In [41]:
print(modelNN.predict(prueba))

[  88038.94869824  650015.17879609  165032.97972511  112034.73383716
  114034.4341103   116034.13438343  118033.83465656  120033.5349297
   87587.48744596   82437.58146824   82437.58146824   87586.90769995
  103034.9131962 ]


In [51]:
#Una prueba mas grande
prueba2 = datos.loc[datos['state_name'] == 'Capital Federal',['lat', 'lon', 'surface_total_in_m2', 'surface_covered_in_m2', 'price_aprox_usd']]
prueba2 = prueba2.loc[prueba2['lat'].notnull(),:]
prueba2 = prueba2.loc[prueba2['lon'].notnull(),:]
prueba2 = prueba2.loc[prueba2['surface_total_in_m2'].notnull(),:]
prueba2 = prueba2.loc[prueba2['surface_covered_in_m2'].notnull(),:]
prueba2 = prueba2.loc[prueba2['price_aprox_usd'].notnull(),:]
prueba2['price_aprox_usd']

14         96000.00
15        120000.00
21        155000.00
25        110000.00
26        110000.00
38        124000.00
39         66000.00
193        73000.00
194        68000.00
203       255200.00
213       170000.00
214       240000.00
232       142500.00
271        99000.00
293        88000.00
295        79900.00
296       140000.00
297        74000.00
314       650000.00
337        79000.00
338        79000.00
371        62000.00
382       990000.00
468        83000.00
489        67900.00
490        69000.00
516       100000.00
536       260000.00
544       140000.00
599        19723.06
            ...    
120072    177000.00
120075    160000.00
120086    220000.00
120195    161500.00
120196    115700.00
120197    126900.00
120198    309200.00
120199    141400.00
120217    170000.00
120229    105000.00
120303    420000.00
120935    265000.00
120963     89000.00
120976     75000.00
120984    147000.00
121057    320000.00
121059     85500.00
121075    315000.00
121077    170000.00


In [56]:
model.predict(prueba2)

array([  96149.5065,  120038.47  ,  154244.27  , ...,  165190.    ,
        165353.69  ,  130939.4932])

In [76]:
(model.predict(prueba2)).size

52304

In [73]:
aver = pd.DataFrame(prueba2.loc[:, 'price_aprox_usd'])
aver['ml'] = model.predict(prueba2)
aver

ValueError: Length of values does not match length of index

In [44]:
#Datos Capital Federal
datosCapital = datos[datos['state_name'] == 'Capital Federal']
datosCapital = datosCapital.loc[datosCapital['price_aprox_usd'].notnull(),:]
datosCapital = datosCapital.loc[datosCapital['rooms'].notnull(),:]
datosCapital = datosCapital.loc[datosCapital['floor'].notnull(),:]
#datosCapitalForTraining = datosCapital[['place_name','price_aprox_usd','rooms','floor']]
datosCapitalForTraining = datosCapital[['price_aprox_usd','rooms','floor']]

In [81]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor


x_train, x_test, y_train, y_test = train_test_split(datosCapitalForTraining, datosCapitalForTraining['price_aprox_usd'], test_size=0.2)

model = RandomForestRegressor(n_estimators=100, max_features='sqrt')
model.fit(x_train,y_train)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [79]:
unoCualquiera = datosCapital[datosCapital['place_name'] == 'Palermo'].head(5)
unoCualquiera = unoCualquiera[['price_aprox_usd','rooms','floor']]
unoCualquiera.head()

,price_aprox_usd,rooms,floor
1010,395000.0,5.0,5.0
1243,190000.0,2.0,3.0
4794,174700.0,2.0,3.0
7492,165000.0,3.0,12.0
8787,520000.0,4.0,5.0


In [82]:
print(model.predict(unoCualquiera))

[ 398700.    189444.88  174620.    165039.96  520800.  ]
